# Regressionsanalyse mit Random Forests

Random Forests wurden Anfang der 1990er als Verbesserung zu einfachen Decision Trees vorgeschlagen. Die Idee dabei ist die Kombinierung vieler einzelner Decision Trees als Ensemble (Forest), um insgesamt eine bessere Vorhersagefähigkeit zu erhalten. Die einzelnen Decision Trees werden unabhängig voneinander trainiert und ihre Vorhersagen am Ende in Form eines finalen Votings (Durchschnittsberechnung bei Regression, Mehrheitsentscheid bei Klassifizierung) zusammengeführt. Daneben werden einige weitere Methoden genutzt, um insgesamt die Modellvorhersagefähigkeit zu steigern:

* Bootstrap Sampling: Jeder Decision Tree erhält ein eigenes Sample aus den Trainingsdaten. Beim Bootstrap Sampling wird ein Anteil der Trainingsdaten zufällig ausgewählt und der Rest wird durch stellvertretende Datenpunkte (z. B. der Durchschnitt des ganzen Samples) aufgefüllt. Das führt dazu, dass die einzelnen Decision Trees über diverse und individuelle Training Sets verfügen. Das gesamte Ensemble wird dadurch robuster, dass die einzelnen Trees einzigartiger werden.
* Feature Randomness: Bei jedem Split von jedem Decision Tree wird nur ein zufällig ausgewählter Teil der Prädiktoren des Datensatzes gesplittet. Auch hier gilt es, die einzelnen Decision Trees möglichst stark zu diversifizieren.
* Growing Trees deep: Die einzelnen Decision Trees werden auf viele Splits (also auf eine höhere Komplexität) trainiert. Dadurch overfitten sie ihre jeweiligen Bootstrap Samples. Da die finale Vorhersage jedoch durch das gesamte Ensemble getroffen wird, wird das Overfitting wieder ausgeglichen.

Das folgende Notebook hat keine Code-Aufgaben und ist optional für alle diejenigen, die Lust haben, ein weiteres Modell kennenzulernen.

## Daten vorbereiten

Bei der Vorbereitung des Datensatzes gehen wir zunächst analog zum Decision Tree Modell vor. 

Wir müssen die Daten wieder laden, aufräumen und Variablen entfernen, die wir nicht als Prädiktoren nutzen wollen. Anschließend teilen wir die Daten in Trainings- und Testdaten auf.

In [ ]:
pacman::p_load(tidyverse, tidymodels, vip)
set.seed(123)

data <- read_csv("data/regression/bike_sharing.csv", show_col_types = FALSE) %>%
    mutate(across(where(is.character), as.factor)) %>%
    janitor::clean_names() %>%
    glimpse()

In [ ]:
data_split <- data %>%
    select(-instant, -dteday) %>%
    select(-casual, -registered) %>%
    initial_split(prop = 0.75)

train_data <- training(data_split)
test_data <- testing(data_split)

## Modell trainieren

Wir können zunächst ein einfaches Random Forest-Modell trainieren. Random Forests sind in `parsnip` über die Funktion [`rand_forest()`](https://parsnip.tidymodels.org/reference/rand_forest.html) implementiert. Ähnlich wie bei `decision_tree()` stehen hier wieder eine Auswahl an Algorithmen zur Verfügung, die wir nutzen können. Der Default-Algorithmus ist die "ranger"-Engine. Wir definieren also wieder unser Modell:

In [ ]:
bikes_model <- rand_forest(
    mode = "regression"            # Build a regression model
    ) %>%
    set_engine(
        "ranger",                  # Using the `ranger` engine
        importance = "permutation" # Include the importance so we can use vi later
    )  

Bei der Wahl der Engine können wir in `set_engine()` dem Argument `importance` den Wert `"permutation"` übergeben. Das bedeutet, dass während des Trainings des Modell die Feature Importance berechnet wird. Damit können wir später mithilfe von `vi` berechnen, welche Prädiktoren den größten Einfluss auf die Zielvariable haben. Prinzipiell ist das ein optionaler Schritt, den wir nur brauchen, wenn wir später die Variable Importance berechnen wollen.

`rand_forest()` hat neben der Engine und dem Modus 3 einstellbare Parameter. Aus der [Dokumentation](https://parsnip.tidymodels.org/reference/rand_forest.html) können wir entnehmen:

* `mtry`: An integer for the number of predictors that will be randomly sampled at each split when creating the tree models. Default value is the square root of the number of predictors.
* `trees`: An integer for the number of trees contained in the ensemble. Default value is 500.
* `min_n`: An integer for the minimum number of data points in a node that are required for the node to be split further. Default value is 5.

### `recipe`

Die Vorbereitung via `recipe` bleibt ebenfalls gleich. Da Random Forests auf einer Vielzahl von Decision Trees beruhen, sind die Grundannahmen beim Pre-Processing die selben. Auch Random Forests sind invariant gegenüber Standardization. Wir können also einfach das Rezept aus dem vorherigen Beispiel übernehmen.

In [ ]:
bikes_recipe <- train_data %>%
    recipe(cnt ~ .) %>%
    step_mutate(across(where(is.logical), as.factor)) %>% # Mutate logical variables to factors
    step_dummy(all_nominal_predictors()) %>%              # Encode categorical variables if needed
    step_impute_mean(all_numeric_predictors()) %>%        # Impute missing values for numeric predictors
    step_corr(all_predictors()) %>%                       # Remove correlating predictors
    step_zv(all_predictors()) %>%                         # Remove zero-variance predictors
    prep()

### `workflows`

Nachdem wir Modell und Rezept fertig definiert haben, können wir wieder einen Workflow erstellen und diesen auf den Trainingsdaten trainieren:

In [ ]:
bikes_workflow <- workflow() %>%
    add_model(bikes_model) %>%
    add_recipe(bikes_recipe)

bikes_fit <- bikes_workflow %>% 
    fit(data = train_data)

## Modell evaluieren

Abschließend können wir das Modell noch evaluieren...

In [ ]:
bikes_fit %>% 
    augment(test_data) %>% 
    metrics(truth = cnt, estimate = .pred)

...und die Variable Importance berechnen:

In [ ]:
bikes_fit %>% 
    extract_fit_parsnip() %>% # extract the model
    vi(scale = TRUE) %>%      # scale the most important variable to 100
    head(6)

Vergleicht die Metriken und die Variable Importance mit dem einfach Decision Tree-Modell. An welchen Stellen hat sich unsere Modellierung verbessert? Was ist jetzt anders?

## Modell tunen

<img src="https://tune.tidymodels.org/logo.png" alt="rsample" width="100" align="right" /> Ein weiterer Schritt zur Modellverbesserung im Machine Learning ist das sogenannte Tuning. Beim Tuning probieren wir verschiedene Modellparameter und ihren Effekt auf Vorhersagefähigkeit und Performance des Modells aus.

Wir erinnern uns, dass die Random Forest-Implementierung in `tidymodels` drei Hyperparameter hatte: `mtry`, `trees` und `min_n`. Diese drei Parameter sind unsere Stellschrauben für das Tuning. Wir können sie manuell verändern und nach jeder Veränderung das Modell trainieren und evaluieren und anschließen vergleichen, welchen Einfluss die Änderung der Hyperparameter auf Vorhersagefähigkeit hat.

Dafür gibt es in `tidymodels` das Paket `tune`. Mit `tune` können wir den Tuning-Prozess komfortabel und automatisiert ausführen. Dazu müssen wir zunächst ein eigenes Modell für das Tuning definieren. In dieser Modelldefinition definieren wir die Hyperparameter nicht manuell, sondern spezifizieren mit der Funktion `tune()`, dass sie später getuned werden sollen:

In [ ]:
tune_model <- rand_forest(
    mode = "regression",            # Build a regression model
    mtry = tune(),
    trees = tune(),
    min_n = tune()
    ) %>%
    set_engine(
        "ranger",                  # Using the `ranger` engine
        importance = "permutation" # Include the importance so we can use vi later
    )  

Dann müssen wir mit dem neuen Modell einen eigenen Workflow definieren - dazu gleich mehr:

In [ ]:
tune_workflow <- workflow() %>%
    add_model(tune_model) %>%
    add_recipe(bikes_recipe)

### V-fold cross-validation

Im Machine Learning wird für das Tuning von Hyperparametern oft eine sogenannte **V-fold** (oder k-fold) **cross-validation** genutzt. Bei diesem Prozess werden die Trainingsdaten in $V$ gleich große Sets aufgeteilt. Bei jedem Validierungsdurchlauf werden $V-1$ Sets als Trainings-Sets benutzt, und ein Set als **Validation Set**.

V-fold cross-validation hilft uns dabei, Overfitting auf bestimmte Hyperparameter zu vermeiden. Außerdem erhalten wir eine robustere Aussage über die Vorhersagefähigkeit und die Performance des Modells. Ebenso ist V-fold cross-validation bei kleinen Datensätzen hilfreich, weil wir durch das Durchtauschen die einzelnen Datenpunkte sowohl für Training als auch Validierung nutzen. 

Aber aufgepasst: ein Validierungsset ersetzt nicht das Test-Set! Dieses sollten wir immer vorher beiseite legen, um eine von den Trainingsdaten unabhängige Evaluierung zu ermöglichen.

In `tidymodels` können wir Validation Sets mithilfe der Funktion `vfold_cv` erstellen. Als typischer Wert bietet sich bspw. $V = 5$ an: 

In [ ]:
cv_folds <- vfold_cv(train_data, v = 5)  # V = 5
cv_folds %>% glimpse()

### Tuning Grid

Das Tuning findet dann auf einem sogenannten Grid statt. Bei drei Hyperparametern können wir uns das Tuning im Prinzip wie ein drei-dimensionales Optimierungsproblem vorstellen. Optimieren bzw. minimieren wollen wir dabei den Root Mean Square Error `rmse`, der eine Aussage darüber trifft, wie gut unsere Modellvorhersage ist. 

`tune` hat verschiene Grid-Funktionen, wir benutzen zunächst einmal `grid_random()`. Für jeden Hyperparameter (also `mtry`, `trees` und `min_n`) können wir eine Spanne einstellen, über die der Hyperparameter getuned werden soll:

* `mtry`: Dieser Parameter beschreibt die Anzahl der zufällig ausgewählten Features bei jedem Split eines Trees. Ein gut funktionierender Wert ist i. d. R. $\sqrt{n}$, wobei $n$ die Anzahl an Prädiktoren ist. Mögliche Werte für `mtry` liegen zwischen 1 und der Anzahl an Prädiktoren.
* `trees`: Dieser Parameter definiert die Anzahl an einzelnen Decision Trees, die das Modell trainiert. Ein Random Forest mit einer niedrigen Anzahl an Trees lässt sich schneller trainieren, wird aber keine so genauen Aussagen treffen können. Ab einer bestimmten Anzahl an Trees wird die Vorhersagefähigkeit des Modells jedoch auch nicht mehr besser, und die *computational costs* für das Training des Modells werden zu groß.
* `min_n`: Dieser Parameter beschreibt die minimale Anzahl an Datenpunkte innerhalb eines Splits. Eine sehr niedrige Zahl führt i. d. R. zu Overfitting, kann aber dabei helfen, komplexe und kleine Cluster innerhalb der Daten zu entdecken. Eine größere Zahl bewirkt das Gegenteil.

Mit `size` bestimmen wir anschließend die Anzahl an verschiedenen, zufälligen Kombinationen, die unser Tuning Grid annehmen soll. Hier lohnt es sich, erst einmal mit einem kleinen Wert zu starten, und sich anschließend auf die Bereiche im Grid zu konzentrieren, die die höchste Vorhersagefähigkeit erzeugen.

In [ ]:
tune_grid <- grid_random(
    mtry(range = c(1, ncol(train_data) - 1)),      
    trees(range = c(200, 1000)), 
    min_n(range = c(2, 40)) ,    
    size = 6               
)

Das eigentliche Tuning wird dann mithilfe der Funktion `tune_grid()` ausgeführt.

An dieser Stelle wird die Sinnhaftigkeit von `workflows` deutlich - das Modell muss ja jedes Mal die Trainingsdaten pre-processen und fitten. Unser Code wird viel übersichtlicher, weil wir diesen Prozess in `tune_workflow` gespeichert haben. Als Trainingsdaten nutzen wir die cross-validation sets, die wir vorher erstellt haben. Für jede Parameter-Kombination auf unserem Tuning Grid werden die Metriken `rmse` und `rsq` berechnet.

Das Tuning kann je nach Serverauslastung 5-10 Minuten dauern:

In [ ]:
tune_results <- tune_grid(
    tune_workflow,
    resamples = cv_folds,
    grid = tune_grid,
    metrics = metric_set(rmse, rsq)
)

### Tuning evaluieren & visualisieren

Jetzt müssen wir das Tuning evaluieren. Dafür können wir die Metriken aus den Tuning-Ergebnissen mithilfe von `collect_metrics()` rausziehen. Wir erhalten eine saubere Tabelle, in der die verschiedenen Parameter-Kombinationen des Tuning Grids und ihre dazugehörigen Metriken abgebildet sind:

In [ ]:
tune_results %>%
    collect_metrics() %>%
    select(mtry, trees, min_n, .metric, mean) %>%
    pivot_wider(
        names_from = .metric,
        values_from = mean
        ) %>%
    arrange(rmse)

Je niedriger der Wert für `rmse`, umso besser ist die Vorhersagefähigkeit des Modells. Wie wir sehen können, hat das Tuning das Modell auch nochmal etwas verbessert, im Vergleich zum einfachen Modellieren ohne Tuning zu Beginn des Notebooks.

In einem echten Anwendungsfall würden wir jetzt das Tuning Grid unserer Hyperparameter noch weiter erkunden, um die bestmögliche Kombination von `mtry`, `trees` und `min_n` zu finden.

### Bestes Modell auswählen...

Aus Zeitgründen wählen wir aber bereits an dieser Stelle das beste Modell aus, und zwar mithilfe von `select_best()`. Mit `finalize_workflow()` definieren wir den finalen Workflow und somit auch das finale Modell...

In [ ]:
best_rf <- tune_results %>%
    select_best(metric = "rmse")

final_rf <- tune_workflow %>%
    finalize_workflow(best_rf)

### ...und fitten

...und fitten das ganze wie gewohnt auf den Trainingsdaten.

Anschließend können wir das finale Modell noch mithilfe der Testdaten evaluieren:

In [ ]:
final_fit <- final_rf %>% fit(data = train_data)

final_fit %>% 
    augment(test_data) %>% 
    metrics(truth = cnt, estimate = .pred)

final_fit %>% 
    extract_fit_parsnip() %>% 
    vi(scale = TRUE) %>%      
    head(6)